In [1]:
import tensorflow as tf
print('Tensorflow is imported.')
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import tensorflow_text as text
import os, pickle5 as pickle
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

2023-05-30 13:45:23.950528: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 13:45:24.827600: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-05-30 13:45:24.827699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64
2023-05-30 13:45:24.827706: W tensorflow/compiler/tf2tensorrt/utils/py_utils

Tensorflow is imported.


In [2]:
def pickler(path, pkl_name, obj):
    with open(os.path.join(path, pkl_name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def unpickler(path, pkl_name):
    with open(os.path.join(path, pkl_name), 'rb') as f:
        obj = pickle.load(f)
    return obj

def get_labels(data):
    labels = []
    for d in data:
        labels.append(d['label']) if d['label'] not in labels else None

    return labels


def get_x_and_y(data):
    x, y = [], []
    for d in data:
        for dd in d['data']:
            tmp = dd['text'].replace('\n', '').replace('_', '')  # clean
            x.append({'label': d['label'], 'dbpedia_uri': dd['dbpedia_uri'], 'context_data': dd['context_data'], 'text': tmp, 'graph': dd['graph']}) if len(tmp) > 0 else None
            y.append(d['label']) if len(tmp) > 0 else None

    return x, y


def get_label_index(label):
    return [index for index, _label in enumerate(unique_labels) if label == _label][0]

def find_clusters(arr):    
    result = []
    for el in arr:
        all_mappings = node_cluster_mapping_with_count[el] if el in node_cluster_mapping_with_count else []
        filtered_mappings = [e[0] for e in all_mappings[:min(len(all_mappings),3)]]
        result.extend(filtered_mappings)
    
    return result

def get_context_data(data):
    for d in data:
        for dd in d['data']:
            dd['context_data'] = [c for c in set(find_clusters(dd['context_graph']['nodes']))]

In [3]:
node_cluster_mapping_with_count= unpickler('data','node_cluster_mapping_v5_with_count.pkl')
data = unpickler('data','classification_data_with_graphs_v5.pkl')


In [4]:
art_labels = ['Art-Painting',
 'Art-Painting-Artist',
 'Art-Sculpting',
 'Art-Sculpting-Artist',
 'Art-Music',
 'Art-Music-Instrument',
 'Art-Cinema',
 'Art-Cinema-Actor',
 'Art-Theatre',
 'Art-Theatre-Actor',
 'Art-Fashion',
 'Art-Fashion-Model',
 'Art-Fashion-Designer',
 'Art-Literature',
 'Art-Literature-Writer',
 'Art-Photography',
 'Art-Photography-Photographer',
 'Art-Dance',
 'Art-Dance-Dancer']

science_labels = ['Science-Biology',
 'Science-Chemistry',
 'Science-Physics',
 'Science-Mathematics',
 'Science-Medicine',
 'Science-Earth Sciences',
 'Science-Agriculture',
 'Science-Archeology',
 'Science-Paleonthology',
 'Science-Social Sciences',
 'Science-Meteorology',
 'Science-Antropology',
 'Science-Economics',
 'Science-Psychology',
 'Science-Politics']

art, science = [d['data'] for d in data if d['label'] in art_labels], [d['data'] for d in data if d['label'] in science_labels]
data2 = [{'label': 'Art', 'data': []},{'label': 'Science', 'data': []}]
for a in art:
    data2[0]['data'].extend(a)
    
for s in science:
    data2[1]['data'].extend(s)

In [5]:
len(data2[0]['data'])

1725

In [ ]:
unique_labels = get_labels(data)

get_context_data(data)

x,y = get_x_and_y(data)

In [6]:
unique_labels = get_labels(data2)

get_context_data(data2)

x,y = get_x_and_y(data2)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
x_val, y_val,x_test, y_test = x_test[:500], y_test[:500], x_test[500:], y_test[500:]
y_train_int, y_test_int = [get_label_index(label) for label in y_train],[get_label_index(label) for label in y_test]
x_train_text,x_val_text,x_test_text = [xx['text'] for xx in x_train],[xx['text'] for xx in x_val], [xx['text'] for xx in x_test]
# x_train_context, x_val_context, x_test_context = [xx['context_data'] for xx in x_train],[xx['context_data'] for xx in x_val], [xx['context_data'] for xx in x_test]

In [8]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

2023-05-30 13:28:44.525090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:28:44.556298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:28:44.556495: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:28:44.557426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', min_delta=0.5)
history = model.fit(x_train_text, y_train_int, epochs=5, batch_size = 16,callbacks=[callback])

Epoch 1/5


2023-05-30 13:29:24.995427: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 24.00MiB (rounded to 25165824)requested by op model/keras_layer_1/StatefulPartitionedCall/StatefulPartitionedCall/StatefulPartitionedCall/bert_encoder/StatefulPartitionedCall/transformer/layer_8/activation/Gelu/mul
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-30 13:29:24.995528: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-05-30 13:29:24.995564: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 106, Chunks in use: 105. 26.5KiB allocated for chunks. 26.2KiB in use in bin. 549B client-requested in use in bin.
2023-05-30 13:29:24.995587: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks 

ResourceExhaustedError: Graph execution error:

failed to allocate memory
	 [[{{node transformer/layer_8/activation/Gelu/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_84632]

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'])
plt.show()
plt.plot(history.history['loss'])
plt.show()

In [15]:
results = model.predict(x_test_text)
def get_result_labels(results):
    return [unique_labels[np.where(row==max(row))[0][0]] for row in results]

result_labels = get_result_labels(results)
print('Accuracy score: %.2f' % accuracy_score(result_labels, y_test))
print(classification_report(y_test, result_labels))

54/54 [==============================] - 46s 854ms/step


NameError: name 'np' is not defined

## FINE TUNING

In [8]:
import tensorflow_models as tfm

/home/cnytync/anaconda3/envs/Python39/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/cnytync/anaconda3/envs/Python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

2023-05-30 13:23:10.405546: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [15]:
tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
    lower_case=True)

In [16]:
max_seq_length = 128

packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_seq_length,
    special_tokens_dict = tokenizer.get_special_tokens_dict())

In [11]:
class BertInputProcessor(tf.keras.layers.Layer):
    def __init__(self, tokenizer, packer):
        super().__init__()
        self.tokenizer = tokenizer
        self.packer = packer

    def call(self, inputs):
        tok1 = self.tokenizer(inputs['text_1'])
        tok2 = self.tokenizer(inputs['text_2'])

        packed = self.packer([tok1, tok2])

        if 'label' in inputs:
            return packed, inputs['label']
        else:
            return packed

bert_inputs_processor = BertInputProcessor(tokenizer, packer)

### Prepare Data

In [12]:
y_train_0_indexes, y_train_1_indexes = [i for i,y in enumerate(y_train_int) if y == 0], [i for i,y in enumerate(y_train_int) if y == 1]
x_train_text_0, x_train_text_1 = [t for i,t in enumerate(x_train_text) if i in y_train_0_indexes], [t for i,t in enumerate(x_train_text) if i in y_train_1_indexes]

limit = 5000
match, no_match = [], [] 

import random
for i in range(0,limit):
    n1 = random.randint(0,len(x_train_text_0) - 1)
    n2 = random.randint(0,len(x_train_text_1) - 1)
    
    no_match.append({'text_1': x_train_text_0[n1], 'text_2': x_train_text_1[n2]})
    
for i in range(0,int(limit / 2)):
    n1 = random.randint(0,len(x_train_text_0) - 1)
    n2 = random.randint(0,len(x_train_text_0) - 1)
    
    match.append({'text_1': x_train_text_0[n1], 'text_2': x_train_text_0[n2]}) if n1 != n2 else None
    
for i in range(0,int(limit / 2)):
    n1 = random.randint(0,len(x_train_text_1) - 1)
    n2 = random.randint(0,len(x_train_text_1) - 1)
    
    match.append({'text_1': x_train_text_1[n1], 'text_2': x_train_text_1[n2]}) if n1 != n2 else None

x_train_ft = []
x_train_ft.extend(match)
x_train_ft.extend(no_match)

y_train_ft = [1 for m in match]
y_train_ft.extend([0 for nm in no_match])

import numpy as np
indexes = np.random.randint(0,len(x_train_ft), len(x_train_ft))

x_train_ft = np.array(x_train_ft)
y_train_ft = np.array(y_train_ft)

x_train_ft = x_train_ft[indexes]
y_train_ft = y_train_ft[indexes]

In [13]:
# Generator function to yield each dictionary
def generator():
    for x,y in zip(x_train_ft,y_train_ft):
        yield {'label':[y], 'text_1': [x['text_1']], 'text_2': [x['text_2']]}

# Create a dataset from the generator
dataset = tf.data.Dataset.from_generator(generator, output_signature={
    'label': tf.TensorSpec(shape=(None,), dtype=tf.int64),
    'text_1': tf.TensorSpec(shape=(None,), dtype=tf.string),
    'text_2': tf.TensorSpec(shape=(None,), dtype=tf.string)
})
dataset = dataset.prefetch(1)
dataset

<PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'text_1': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'text_2': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>

In [14]:
x_train = dataset.map(bert_inputs_processor).prefetch(1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


### Tensorflow Model

In [15]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())
config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [16]:
encoder_config = tfm.nlp.encoders.EncoderConfig({
    'type':'bert',
    'bert': config_dict
})
bert_encoder = tfm.nlp.encoders.build_encoder(encoder_config)

In [17]:
bert_classifier = tfm.nlp.models.BertClassifier(network=bert_encoder, num_classes=2)

In [18]:
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
checkpoint.read(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

### Learning rate

In [19]:
# Set up epochs and steps
epochs = 5
batch_size = 32
eval_batch_size = 32

train_data_size = len(x_train_ft)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(0.1 * num_train_steps)
initial_learning_rate=2e-5

In [20]:
linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=0,
    decay_steps=num_train_steps)

In [21]:
warmup_schedule = tfm.optimization.lr_schedule.LinearWarmup(
    warmup_learning_rate = 0,
    after_warmup_lr_sched = linear_decay,
    warmup_steps = warmup_steps
)

In [22]:
optimizer = tf.keras.optimizers.experimental.Adam(
    learning_rate = warmup_schedule)

In [23]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

### Fine-tune Fit

In [25]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', min_delta=0.5)
history = bert_classifier.fit(
      x_train,
      batch_size=32,
      epochs=5, callbacks=[callback])

Epoch 1/5
9998/9998 [==============================] - 1341s 134ms/step - loss: 0.2412 - accuracy: 0.9134
Epoch 2/5
9998/9998 [==============================] - 1331s 133ms/step - loss: 0.2299 - accuracy: 0.9226


In [28]:
my_examples = {
        'text_1':[
            'The rain in Spain falls mainly on the plain.',
            'Look I fine tuned BERT.'],
        'text_2':[
            'It mostly rains on the flat lands of Spain.',
            'Is it working? This does not match.']
    }

In [29]:
ex_packed = bert_inputs_processor(my_examples)
my_logits = bert_classifier(ex_packed, training=False)

result_cls_ids = tf.argmax(my_logits)
result_cls_ids

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 1])>

### Export the model

In [39]:
class ExportModel(tf.Module):
    def __init__(self, input_processor, classifier):
        self.input_processor = input_processor
        self.classifier = classifier

    @tf.function(input_signature=[{
      'text_1': tf.TensorSpec(shape=[None], dtype=tf.string),
      'text_2': tf.TensorSpec(shape=[None], dtype=tf.string)}])

    def __call__(self, inputs):
        packed = self.input_processor(inputs)
        logits =  self.classifier(packed, training=False)
        result_cls_ids = tf.argmax(logits)
        return {
            'logits': logits,
            'class_id': result_cls_ids,
            'class': tf.gather(
                tf.constant(unique_labels),
                result_cls_ids)
        }

export_model = ExportModel(bert_inputs_processor, bert_classifier)

In [40]:
import tempfile
export_dir=tempfile.mkdtemp(suffix='_saved_model')
tf.saved_model.save(export_model, export_dir=export_dir,
                    signatures={'serving_default': export_model.__call__})

INFO:tensorflow:Assets written to: /tmp/tmpqmivj2at_saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmpqmivj2at_saved_model/assets


### Fit train 

In [12]:
export_dir = '/tmp/tmpqmivj2at_saved_model'
bert_classifier = tf.saved_model.load(export_dir)

2023-05-30 13:22:39.051926: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:22:39.176448: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:22:39.176832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:22:39.192370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [11]:
import tensorflow_models as tfm

gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

tokenizer = tfm.nlp.layers.FastWordpieceBertTokenizer(
    vocab_file=os.path.join(gs_folder_bert, "vocab.txt"),
    lower_case=True)

max_seq_length = 128

packer = tfm.nlp.layers.BertPackInputs(
    seq_length=max_seq_length,
    special_tokens_dict = tokenizer.get_special_tokens_dict())

2023-05-30 13:46:54.821935: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".
2023-05-30 13:46:56.781597: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:46:56.814275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-30 13:46:56.814581: I tensorflow/compiler/xla/stream_exe

In [12]:
class BertInputProcessor(tf.keras.layers.Layer):
    def __init__(self, tokenizer, packer):
        super().__init__()
        self.tokenizer = tokenizer
        self.packer = packer

    def call(self, inputs):
        tok1 = self.tokenizer(inputs['text'])

        packed = self.packer([tok1])

        if 'label' in inputs:
            return packed, inputs['label']
        else:
            return packed

bert_inputs_processor_ft_train = BertInputProcessor(tokenizer, packer)

In [13]:
# Generator function to yield each dictionary
def generator():
    for x,y in zip(x_train_text,y_train_int):
        yield {'label':[y], 'text': [x]}

# Create a dataset from the generator
dataset = tf.data.Dataset.from_generator(generator, output_signature={
    'label': tf.TensorSpec(shape=(None,), dtype=tf.int64),
    'text': tf.TensorSpec(shape=(None,), dtype=tf.string),
})
dataset = dataset.prefetch(1)
dataset

<PrefetchDataset element_spec={'label': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'text': TensorSpec(shape=(None,), dtype=tf.string, name=None)}>

In [14]:
x_train = dataset.map(bert_inputs_processor_ft_train).prefetch(1)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [16]:
export_dir = '/tmp/tmpqmivj2at_saved_model'
bert_classifier = tf.saved_model.load(export_dir)

In [18]:
predictions = bert_classifier(x_train_text,y_train_int)

TypeError: f(inputs) has an input_signature ({'text_1': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'text_2': TensorSpec(shape=(None,), dtype=tf.string, name=None)},) which specifies 1 positional arguments, but got 2.

In [45]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode='max', min_delta=0.5)
history = bert_classifier.fit(
      x_train,
      batch_size=32,
      epochs=5, callbacks=[callback])

Epoch 1/5
2051/2051 [==============================] - 283s 137ms/step - loss: 1.5032 - accuracy: 0.4564
Epoch 2/5
2051/2051 [==============================] - 273s 133ms/step - loss: 1.5120 - accuracy: 0.4520
